In [175]:
import os
import sys
import re
import json
import numpy as np
import matplotlib.pyplot as plt

In [10]:
with open("test_statblock/apiResponse.json", 'r') as f:
    resp = json.load(f)

In [183]:
def filter_blocks(blocks, threshold=0.15):
    columns = []
    column_points = []
    column_widths = []

    top_block = None
    bottom_block = None

    tops = []
    lefts = []
    for i,block in enumerate(blocks):
        if block["BlockType"] != "LINE":
            continue
        tops.append((i, block["Geometry"]["BoundingBox"]["Top"]))
        lefts.append((i, block["Geometry"]["BoundingBox"]["Left"]))

    tops.sort(key=lambda x: x[1])
    lefts.sort(key=lambda x: x[1])

    tdists = []
    ldists = []
    for i in range(1, len(tops)):
        tdists.append(tops[i][1] - tops[i-1][1])
        ldists.append(lefts[i][1] - lefts[i-1][1])
    
    #plt.plot(tdists)
    #print(ldists)

    ldists = np.array(ldists)
    tdists = np.array(tdists)

    print(np.array(ldists).var())

    for i in range(len(ldists)):
        if ldists[i] > ldists.mean() + np.sqrt(ldists.var()):
            print('L', blocks[lefts[i][0]]["Text"], blocks[lefts[i+1][0]]["Text"])
        if tdists[i] > tdists.mean() + np.sqrt(tdists.var()):
            print('T', blocks[tops[i][0]]["Text"], blocks[tops[i+1][0]]["Text"])
    #print(lefts)

filter_blocks(resp["Blocks"])

0.00018154654231734838
T Artist credit: Cze & Peku THE LORD OF THE DEEP
L STR DEX
L DEX CON
L CON INT
L Monster stats written by Dagonzo Adventures. Art by Cze & Peku. WIS
L WIS CHA
L CHA Attack. The Lord of the Deep makes one bite, claw, or
L 14 (2d8+5) piercing damage plus 7 (2d6) necrotic Artist credit: Cze & Peku
L Artist credit: Cze & Peku -
T to attack an adjacent enemy. Monster stats written by Dagonzo Adventures. Art by Cze & Peku.


In [153]:
def blocks_to_joined_text(blocks, threshold=0.15):
    columns = []
    column_points = []
    column_widths = []

    top_block = None
    bottom_block = None

    tops = []
    lefts = []
    for i,block in enumerate(blocks):
        if block["BlockType"] != "LINE":
            continue
        tops.append((i, block["Geometry"]["BoundingBox"]["Top"]))
        lefts.append((i, block["Geometry"]["BoundingBox"]["Left"]))

    tops.sort(key=lambda x: x[1])
    lefts.sort(key=lambda x: x[1])

    for block in blocks:
        #Only look at 'Line types
        if block["BlockType"] != "LINE":
            continue
      
        #Filter out any spurious blocks without muc text
        if "Text" not in block or len(block["Text"]) < 2:
            continue

        #Get top left position
        box = block["Geometry"]["BoundingBox"]
        pos = box["Left"]

        if not top_block:
            top_block = box["Top"]

        print(box["Top"], block["Text"])

        if box["Top"] < top_block:
            continue

        #Check if this fits within an existing column
        written = False
        for i,p in enumerate(column_points):
            #print(p, p + column_widths[i], pos)
            if pos > (p-0.01) and (pos - p) < (column_widths[i]*1.1):
                written = True
                columns[i].append((box, block["Text"]))
                break
        if not written:
            columns.append([(box, block["Text"])])
            column_points.append(pos)
            column_widths.append(box["Width"])

    columns.sort(key=lambda x: x[0][0]["Left"])

    last = {"Left":0, "Top":0}
    boxes = []
    threshold = 0.015
    current_box = []

    top = columns[0][0][0]["Top"]

    for col in columns:
        for c in col:
            x_dist = last["Left"] - c[0]["Left"]
            y_dist = c[0]["Top"] - last["Top"]

            if c[0]["Top"]  - top < 0:
                continue
            #print(c, y_dist)
            last = c[0]
            if y_dist <= threshold:
                current_box.append(c[1])
            else:
                if len(current_box) > 0:
                    boxes.append([x for x in current_box])
                current_box = [c[1]]
            #print(c[1], round(x_dist,3), round(y_dist,3))
    boxes.append(current_box)

    return boxes

In [154]:

class Creature(object):

    def set_size(self, s):
        self.size = s.strip().upper()[0]

    def set_name(self, n):
        ws = [w.lower() for w in n.split()]
        ws = [w[0].upper() + w[1:] if w not in ["in", "of", "the", "and", "a", "an"] or i == 0 else w for i,w in enumerate(ws) ]
        self.name = " ".join(ws)

    def set_type(self, t):
        t = re.sub('[^\w\s\d]','', t)
        if t not in CREATURE_TYPES:
            print("ERROR parsing creature type {}".format(t))
            return None
        self.type = t
        return self.type

    def set_source(self, s):
        self.source = s

    def set_alignment(self, alignment_string):
        align = []
        lc = ge = ""
        parts = [p.strip() for p in alignment_string.upper().split()]
        if parts[0] == "ANY":
            if len(parts) == 1:
                align = ['A']
            elif parts[1] == "CHAOTIC" or parts[1] == 'LAWFUL' :
                align = [parts[1][0], 'G', 'NY', 'E']
            elif parts[1] == 'GOOD' or parts[1] == 'EVIL':
                align = ['L', 'NX', 'C', parts[1][0]]
            elif parts[1] == 'NEUTRAL':
                align = ['NX', 'NY', 'N']
            elif parts[1].startswith('NON'):
                if parts[1] == "NON-CHAOTIC":
                    align = ['NX', 'L', 'G', 'NY', 'E']
                elif parts[1] == 'NON-LAWFUL' :
                    align = ['C', 'NX', 'G', 'NY', 'E']
                elif parts[1] == 'NON-GOOD':
                    align = ['L', 'NX', 'C', 'NY', 'E']
                elif parts[1] == 'NON-EVIL':
                    align =  ["L", "NX","C", "NY", "G"]
        elif parts[0] == 'UNALIGNED':
            align = ['U']
        else:
            if len(parts) == 1:
                align = [parts[0][0]]

            else:
                align = [parts[0][0], parts[1][0]]

        if len(align) == 0:
            print("ERROR parsing alignment string {}".format(alignment_string))
            return None

        self.align = align
        return self.align

    def set_ac(self, ac_string):
        match = re.compile('([0-9]+)\s*(?:\((.*)\))?')
        m = match.match(ac_string)
        groups = m.groups()
        if groups[0] == None:
            print("ERROR: Failed to parse AC string {}".format(ac_string))
            return None

        if groups[1]:
            self.ac = [{
                "ac":int(groups[0]),
                "from": [g.strip() for g in groups[1].split(",")]
            }]
        else:
            self.ac = [int(groups[0])]

        return self.ac
        
    def set_hp(self, hp_string):
        match = re.compile("([0-9]+)\s*\+?\s*(?:\(([0-9]+d[0-9]+\s*\+\s*[0-9]*)\))?")
        m = match.match(hp_string)
        groups = m.groups()
        self.hp = {}

        if groups[0] == None:
            print("ERROR parsing HP string = {}".format(hp_string))
            return None
        
        self.hp["average"] = int(groups[0])

        if groups[1] != None:
            self.hp = {
                "formula": groups[1],
                "average": int(groups[0])
            }
        else:
            self.hp = {
                "special": int(groups[0])
            }

        return self.hp

    def set_speed(self, speed_string):
        walk_match = re.compile('^([0-9]+)\s*ft\.?')
        m = walk_match.match(speed_string)
        speed = {}
        if m:
            speed["walk"] = int(m.groups()[0])
        
        other_match = re.compile('(\w+) ([0-9]+)\s*ft\.?')
        m = other_match.findall(speed_string)
        for res in m:
           speed[res[0]] = int(res[1])

        self.speed = speed
        return self.speed

    def set_attributes(self, attr_string):
        parts = attr_string.split()
        abs = ["str", "dex", "con", "int", "wis", "cha"]
        if len(parts) != 12:
            print("ERROR parsing attribute string {}".format(attr_string))
            return None

        for i,a in enumerate(abs):
            self.attributes[a] = parts[2*i]

        return self.attributes

    def set_saves(self, save_string):
        saves = [s.strip() for s in save_string.split(",")]
        parsed_saves = {}
        match = re.compile("([a-z]{3})\s*([+-][0-9]+)")
        for s in saves:
            m = match.match(s.lower())
            if not m:
                print("ERROR parsing save string {}".format(save_string))
                return None
            gs = m.groups()
            parsed_saves[gs[0]] = gs[1]

        self.saves = parsed_saves        

    def set_skills(self, skill_string):
        skills = [s.strip() for s in skill_string.split(",")]
        parsed_skills = {}
        match = re.compile("([a-z]+)\s*([+-][0-9]+)")
        for s in skills:
            m = match.match(s.lower())
            if not m:
                print("ERROR parsing skill string {}".format(skill_string))
                return None
            gs = m.groups()
            parsed_skills[gs[0]] = gs[1]

        self.skills = parsed_skills

    def set_resistances(self, res_string):
        tmp = res_string.split(";")
        reses = [*[t.strip() for t in tmp[0].split(',')], *[t.strip() for t in tmp[1:]]]
        resistances = []
        for r in reses:
            if len(r.split()) == 1:
                resistances.append(r.lower().strip())
            else:
                pre_note = []
                post_note = []
                res = []
                in_pre = True
                in_res = False
                r_strip = re.sub('[^\w\s\d]','', r).split()
                for i,w in enumerate(r.split()):
                    if r_strip[i].lower().strip() in DAMAGE_TYPES:
                        in_res=True
                        in_pre=False
                        res.append(r_strip[i].lower().strip())
                    elif in_pre:
                        pre_note.append(w)
                    elif not in_res:
                        post_note.append(w)
                    else:
                        if w != "and":
                            in_res=False
                            post_note.append(w)

                resistances.append(
                    {"resist":res}
                )
                if len(pre_note) > 0:
                    resistances[-1]["preNote"] = " ".join(pre_note)
                if len(post_note) > 0:
                    resistances[-1]["note"] = " ".join(post_note)

        self.resistances = resistances

    def set_senses(self, sense_string):
        senses = [s.strip() for s in sense_string.split(",")]
        parsed_senses = []
        tags = []
        match = re.compile("[\w\s]+\s[0-9]+")
        for s in senses:
            if "passive" in s:
                self.passive_perception = int(s.split()[-1])
            else:
                parsed_senses.append(s)
                if "dark" in s:
                    tags.append("D")
                elif "blind" in s:
                    tags.append('B')
                elif "tremor" in s:
                    tags.append('T')
                elif "true" in s:
                    tags.append("U")

        self.senses = parsed_senses
        self.senseTags = tags
        return self.senses

    def set_languages(self, lang_string):
        self.languages = [l.strip() for l in lang_string.split(",")]
        return self.languages

    def set_cr(self, cr_string):
        match = re.compile("([0-9]+)\s+\(.*?XP\s*\)")
        crs = match.findall(cr_string)
        if len(crs) == 1:
            self.cr = int(crs[0])
        else:
            cr = {"cr":int(crs[0])}
            if "lair" in cr_string:
                cr["lair"] = int(crs[1])
            elif "coven" in cr_string:
                cr["coven"] = int(crs[1])
            else:
                print("ERROR failed to parse CR {}".format(cr_string))
                return None

            self.cr = cr
        
        return self.cr

    def add_trait(self, trait):
        ws = trait.split()
        title = None
        for i,w in enumerate(ws):
            if w[-1] == '.':
                title = " ".join(ws[:i+1])
                body = " ".join(ws[i+1:])
                break

        if title is None:
            print("ERROR failed to parse trait {}".format(trait))

        trait = {
            "name":title,
            "entries":body
        }
        self.traits.append(trait)

    def to_json(self):
        return {
            "name":self.name,
            "size":self.size,
            "type":self.type,
            "source":self.source,
            "alignment":self.align,
            "ac":self.ac,
            "hp": self.hp,
            "speed":self.speed,
            **self.attributes,
            "passive":self.passive_perception,
            "cr":self.cr,
            "senses":self.senses,
            "senseTags":self.senseTags,
            "save":self.saves,
            "skill":self.skills,
            "resist":self.resistances,
            "languages":self.languages,
            "traits":self.traits
        }

    def __init__(self):
        self.name = ""
        self.size = ""
        self.type = ""
        self.source = ""
        self.align = []
        self.ac = []
        self.hp = {}
        self.speed = {}
        self.attributes = {}
        self.passive_perception = []
        self.cr = {}
        self.senses = []
        self.senseTags = []
        self.saves = {}
        self.skills = {}
        self.resistances = {}
        self.languages = []
        self.traits = []


In [155]:
states = {
    0:"title",
    1:"topline",
    2:"defence",
    3:"saves",
    4:"features",
    5:"actions",
    6:"legendary_actions"
}

SIZES = [
    "tiny",
    "small",
    "medium",
    "large",
    "huge",
    "gargantuan"
]

def defence_joiner(ws):
    if ws[0].lower() == "saving" and ws[1].lower() == "throws":
        return 2
    if ws[0].lower() == "skills":
        return 1
    if ws[0].lower() == "damage":
        if ws[1].lower() == "resistances":
            return 2
        if ws[1].lower() == "immunities":
            return 2
        if ws[1].lower() == "vulnerabilities":
            return 2
    if ws[0].lower() == "condition" and ws[1].lower() == "immunities":
        return 2
    if ws[0].lower() == "senses":
        return 1
    if ws[0].lower() == "languages":
        return 1
    if ws[0].lower() == "challenge":
        return 1

    return -1

def titled_joiner(ws):
    print(ws)
    for i in range(min(5, len(ws)-3)):
        if ws[i][-1] == '.':
            return i+1
    return -1

def box_join(box, is_start_func):
    new_lines = []
    for line in box:
        ws = line.split()
        split_point = is_start_func(ws)
        if split_point == -1:
            new_lines[-1][1] += " " + line
        else:
            new_lines.append([" ".join(ws[:split_point]), " ".join(ws[split_point:])])
    return new_lines


def parse_joined_text(jt):
    c = Creature()

    state = 0
    for box in jt:
        #Handle title
        if state == 0:
            c.set_name(" ".join(jt[0]))
            state = 1
            continue

        elif state == 1:
            for line in box:
                words = [w.lower() for w in line.split()]
                #Handle first line
                if words[0] in SIZES:
                    c.set_size(words[0])
                    c.set_type(words[1])
                    c.set_alignment(" ".join(words[2:]))

                if words[0] == "armor":
                    c.set_ac(" ".join(line.split()[2:]))
                elif words[0] == "hit":
                    c.set_hp(" ".join(line.split()[2:]))
                elif words[0] == "speed":
                    c.set_speed(" ".join(line.split()[1:]))
                    state = 2
                    continue

        elif state == 2:
            if box[0] == "STR":
                continue
            else:
                c.set_attributes(box[0])
                state = 3
                continue

        elif state == 3:
            box = box_join(box, defence_joiner)
            for key, line in box:
                if key.lower() == "saving throws":
                    c.set_saves(line)
                elif key.lower() == "skills":
                    c.set_skills(line)
                elif key.lower() == "damage resistances":
                    c.set_resistances(line)
                elif key.lower() == "senses":
                    c.set_senses(line)
                elif key.lower() == "languages":
                    c.set_languages(line)
                elif key.lower() == "challenge":
                    c.set_cr(line)
            state = 4
            continue

        elif state == 4:
            box = " ".join(box)
            if box == "Actions":
                state = 5
                continue
            c.add_trait(box)
            
    return c

jt = blocks_to_joined_text(resp["Blocks"])
#for r in jt:
#    print(r)
c = parse_joined_text(jt)
print(c.to_json())
with open("monster.json", 'w') as f:
    f.write(json.dumps(c.to_json()))

0.23932631313800812 Artist credit: Cze & Peku
0.2783324420452118 THE LORD OF THE DEEP
0.28197064995765686 Sunlight Hypersensitivity. The Lord of the Deep takes
0.3003185987472534 Medium monstrosity, chaotic evil
0.29420438408851624 20 radiant damage when it starts its turn in sunlight.
0.3067009747028351 While in sunlight, it has disadvantage on attack rolls
0.3185965418815613 and ability checks.
0.33086836338043213 Armor Class 18 (natural armor)
0.3427649438381195 Hit Points 144 (17d8+68)
0.3465016186237335 Actions
0.3550455570220947 Speed 10 ft., swim 40 ft.
0.36818262934684753 Multiattack. The Lord of the Deep makes two attacks:
0.37995028495788574 one with its bite and one with its claws or Trident of
0.39036452770233154 STR
0.3902231752872467 DEX
0.3902980089187622 CON
0.39028164744377136 INT
0.3902583420276642 WIS
0.39015066623687744 CHA
0.3931317627429962 the Deep.
0.4088931679725647 18 (+4) 18 (+4) 18 (+4) 17 (+3) 15 (+2) 18 (+4)
0.4138546288013458 Bite. Melee Weapon Attack: +9

In [119]:
tests = {
    "ac": [
        ("15", [15]),
        ("17 (plate armour, shield)", [{"ac":17, "from":["plate armour", "shield"]}]),
        ("21 (heavy armour of Arenthis + 1)", [{"ac":21, "from":["heavy armour of Arenthis + 1"]}])
    ],
    "hp": [
        ("15", {"special":15}),
        ("42 (5d12 + 10)", {"formula":"5d12 + 10", "average":42})
    ],
    "speed": [
        ("10 ft.", {"walk":10}),
        ("30ft., swim 10ft", {"walk":30, "swim":10}),
        ("10 ft., swim 40 ft., burrow 10ft", {"walk":10, "swim":40, "burrow":10}),
        ("fly 60ft", {"fly":60})
    ]
}
c = Creature()
for t in tests:
    if t == "ac":
        for case in tests[t]:
            if case[1] != c.set_ac(case[0]):
                print("Failed case {} - returned {}".format(case, c.set_ac(case[0])))
    if t == "hp":
        for case in tests[t]:
            if case[1] != c.set_hp(case[0]):
                print("Failed {} case {} - returned {}".format(case, c.set_hp(case[0])).format(t))
    if t == "speed":
        for case in tests[t]:
            if case[1] != c.set_speed(case[0]):
                print("Failed case {} - returned {}".format(case, c.set_speed(case[0])))

In [100]:
c.set_cr('8 (3,900 XP) or 11 (7,200 XP) when encountered in lair')

{'cr': 8, 'lair': 11}

In [164]:

m = match.match('71354642+ (3d1+    100)')
m.groups()

('71354642', None)

In [101]:
match.groups()

TypeError: 'int' object is not callable